In [ ]:
!pip install onnxruntime

In [ ]:
from transformers import AutoFeatureExtractor
import onnxruntime, pickle
from scipy.special import softmax
from PIL import Image

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224",
                                                        do_rescale=False)

In [4]:
img_size = 224

In [ ]:
with open('/path/to/classes.pkl', 'rb') as f:
    classes = pickle.load(f)

In [ ]:
filepath = """onnx file path"""
ort_session = onnxruntime.InferenceSession(filepath)
input_name = ort_session.get_inputs()[0].name

In [ ]:
def inference(img_path):

    img = Image.open(img_path)
    img_resized = img.resize((224, 224))
    encoding = feature_extractor(images=img_resized, return_tensors='pt')

    ort_inputs = {input_name: encoding['pixel_values'].numpy()}
    ort_outs = ort_session.run(None, ort_inputs)

    result = softmax(ort_outs)
    result_dict = {}

    for i, value in enumerate(result[0][0].tolist()):
        result_dict[classes[str(i)]] = value
        result = sorted(result_dict.items(), key=lambda x:x[1], reverse=True)

    return result

In [ ]:
result = inference(img_path)

for i, (k, v) in enumerate(result[:3]):
    print(f"{k}일 확률 {int(v*100)}%")